## **MASTERGIS: Clasificacion de Imagenes Satelitales**

En esta lectura, aprenderemos sobre:

- **Clasificacion supervisada**.
- **Clasificacion no supervisada**.
<center>
<img src='https://user-images.githubusercontent.com/16768318/73627817-0a23e100-4646-11ea-83f4-07293b85c4d9.png' width = 50% align="right">
 
<img src='https://user-images.githubusercontent.com/16768318/73627849-2aec3680-4646-11ea-9dd3-b52b13652833.gif' width = 30% align="left">
</center>

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"1//05P739font3A1CgYIARAAGAUSNwF-L9IrQLJR51eiG045wEQTaEY58dcZqNDMR9DX_Kkh_guasDrd8dvft2a7AXhfq1hBk20FCLA"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **1. Clasificación supervisada**
El modulo **ee.Classifier** maneja la clasificación supervisada en Earth Engine. El flujo de trabajo general para la clasificación es:

1. Recopilar datos de entrenamiento. Reuna puntos que tienen como propiedad el **"target"** y las variables predictoras.

2. Instanciar un clasificador. Establezca sus parametros si es necesario.

3. Entrene al clasificador utilizando los datos de entrenamiento.

4. Clasificar una imagen o FeatureCollection.
5. Estime el error de clasificacion con datos de validacion independientes.


Los datos de entrenamiento siempre deben definirse como un **ee.FeatureCollection** con una propiedad que almacene el target y propiedades que almacenen las variables predictoras. El **target** debe ser enteros pequeños, consecutivos y no negativos que representen clases. Si es necesario, utilice **remap()** para convertir los valores de clase en enteros consecutivos a partir de 0. Los predictores deben ser numéricos.

Obtenga un clasificador de uno de los constructores en **ee.Classifier**. Entrene al clasificador usando **classifier.train()**. Clasifique una imagen o FeatureCollection usando **classify()**. El siguiente ejemplo utiliza un clasificador de arboles de clasificacion y regresion (CART) (Breiman et al. 1984) para predecir areas forestales y no forestales en la Amazonia:

In [0]:
# Cargar puntos de entrenamiento. La propiedad numerica 'clase' almacena etiquetas conocidas
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# Las imagenes de entrada son un compuesto Landsat 8 sin nubes.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
       .filterBounds(points)

image = ee.Algorithms.Landsat.simpleComposite(
  collection = l8.filterDate('2018-01-01', '2018-12-31'),
  asFloat =  True
)

# Use estas bandas para la prediccion.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

**Visualizemos los datos de entrada**

In [0]:
center = points.geometry().centroid().coordinates().getInfo()
print(points.first().getInfo())
mapdisplay(center,{'train_dataset':points.getInfo(),
                   'image':image.getMapId({'bands':['B5','B4','B3'],'gamma':1.4})})

**Clasificar una imagen landsat 8 usando arboles de decision (CART)**

In [0]:
# Esta propiedad de la tabla almacena el target (cobertura del suelo).
label = 'landcover'

# Superponga los puntos en las imagenes para obtener los datos de entrenamiento.
training = image.select(bands).sampleRegions(
  collection = points,
  properties = [label],
  scale = 30)

# Entrena un clasificador CART con parámetros predeterminados.
trained = ee.Classifier.cart().train(training, label, bands)

# Clasifique la imagen con las mismas bandas utilizadas para el entrenamiento.
classified = image.select(bands).classify(trained)

In [43]:
mapdisplay(center,{'train_dataset':points.getInfo(),
                   'image':image.getMapId({'bands':['B5','B4','B3'],'gamma':1.4}),
                   'clasificacion':classified.getMapId({'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']})})

In [42]:
train_accuracy = trained.confusionMatrix()

print('Validation error matrix: ', train_accuracy.getInfo())
print('Validation overall accuracy: ', train_accuracy.accuracy().getInfo())

Validation error matrix:  [[32, 0, 0], [0, 32, 0], [0, 0, 34]]
Validation overall accuracy:  1


Este siguiente ejemplo utiliza un clasificador **Random Forest** con 10 arboles ([Breiman 2001](https://rd.springer.com/article/10.1023/A:1010933404324). Para evaluar la precisión de un clasificador, utilizamos una **matriz de confusion**(Stehman 1997). 

In [0]:
# Las imagenes de entrada son un compuesto Landsat 8 sin nubes.
center = [-62.836, -9.2399]
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
       .filterBounds(ee.Geometry.Point(center))

image = ee.Algorithms.Landsat.simpleComposite(
  collection = l8.filterDate('2018-01-01', '2018-12-31'),
  asFloat = True
)

# Use estas bandas para la prediccion.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11'];

# Poligonos creados manualmente.
forest1 = ee.Geometry.Rectangle(-63.0187, -9.3958, -62.9793, -9.3443)
forest2 = ee.Geometry.Rectangle(-62.8145, -9.206, -62.7688, -9.1735)
nonForest1 = ee.Geometry.Rectangle(-62.8161, -9.5001, -62.7921, -9.4486)
nonForest2 = ee.Geometry.Rectangle(-62.6788, -9.044, -62.6459, -8.9986)

# Hacer un FeatureCollection de las geometrias hechas a mano.
polygons = ee.FeatureCollection([
  ee.Feature(nonForest1, {'class': 0}),
  ee.Feature(nonForest2, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
  ee.Feature(forest2, {'class': 1}),
])

# Obtenga los valores para todos los pixeles en cada poligono en el entrenamiento.
# Obtenga la muestra de los poligonos FeatureCollection.
# Mantenga esta lista de propiedades de los poligonos.
# Establezca la escala para obtener pixeles Landsat en los poligonos.
training = image.sampleRegions(
  collection = polygons,  
  properties = ['class'],  
  scale = 30
)

# Crear un clasificador RF con parámetros personalizados.
classifier = ee.Classifier.randomForest(10)

# Entrenar el clasificador
trained = classifier.train(training, 'class', bands)

# Clasificar la imagen
classified = image.classify(trained)

In [12]:
mapdisplay(center, {'composite':image.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.5, 'gamma': 2},),
                    'deforestation': classified.getMapId({'min': 0, 'max': 1, 'palette': ['red', 'green']}),
                    'polygon':polygons.getInfo()})

In [0]:
train_accuracy = training.classify(trained)

# Get a confusion matrix representing expected accuracy.
trainAccuracy = train_accuracy.errorMatrix('class', 'classification')
print('Validation error matrix: ', trainAccuracy.getInfo())
print('Validation overall accuracy: ', trainAccuracy.accuracy().getInfo())


### **2. Clasificacion no supervisada (clustering)**

El modulo **ee.Clusterer** maneja la clasificación no supervisada (o agrupamiento) en Earth Engine. Estos algoritmos se basan actualmente en los algoritmos con el mismo nombre en [Weka](https://www.cs.waikato.ac.nz/ml/weka/).

Los clústeres se usan de la misma manera que los **clasificadores** en Earth Engine. El flujo de trabajo general para la agrupación es el siguiente:

1. Ensamblar entidades con propiedades numericas en las que encontrar clusteres.

2. Instanciar un clusterer. Establezca sus parametros si es necesario.

3. Entrene al clusterer usando los datos de entrenamiento.

4. Aplique el clusterer a una colección de imagenes o caracteristicas.

5. Ponga nombre a los clusters.

Los datos de entrenamiento son un **ee.FeatureCollection** con propiedades que se ingresaran al clusterer. A diferencia de los clasificadores, **no hay un valor de clase de entrada para un ee.Clusterer**. Al igual que los clasificadores, se espera que los datos para el entrenamiento y los pasos de aplicacion tengan el mismo numero de valores. Cuando se aplica un cluster entrenado a una imagen o tabla, asigna una ID de cluster entero a cada pixel o feature.

In [0]:
# Load a pre-computed Landsat composite for input.
Input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

# Define a region in which to generate a sample of the input.
region = ee.Geometry.Rectangle(29.7, 30, 32.5, 31.7)
region_img = ee.Image().paint(region, 0, 2)
# Display the sample region.
center = [31.0,31.5]
Mapdisplay(center,{'Region':region_img.getMapId()},zoom_start=8)

In [0]:
# Make the training dataset.
training = Input.sample(
  region=region,
  scale=30,
  numPixels=5000
)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = Input.cluster(clusterer)
result_tk = result.randomVisualizer().getMapId()

In [0]:
# Display the clusters with random colors.
mapdisplay(center, {'results':result_tk},zoom_start=8)